In [138]:
!pip install keras-tuner --upgrade

In [139]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import re
from utils import *
from layers import *
import keras_tuner
from sklearn.preprocessing import RobustScaler, QuantileTransformer, PowerTransformer


In [140]:
nrows = 150000
x_train_df = pd.read_csv("input_training.csv", index_col="ID")
y_train_df = pd.read_csv("output_training_gmEd6Zt.csv", index_col="ID")
x_test_df = pd.read_csv("input_test.csv", index_col="ID")

if nrows is not None:
    x_train_df = x_train_df.iloc[-nrows:]
    y_train_df = y_train_df.iloc[-nrows:]

y_train_df["reod"] = y_train_df["reod"] + 1

train_df = x_train_df.join(y_train_df).copy(deep=True)

# Cleaning Data
1 - Dealing with NaNs

In [141]:
# Check distribution of label on row that contain NaN
train_df.loc[(train_df.isna().sum(axis=1) > 10), "reod"].value_counts()

1    12470
0     3355
2     2969
Name: reod, dtype: int64

In [142]:
#As NaNs are also present in the test dataset, I cannot delete. I chose to replace them with zeros
train_df.replace(np.nan, 0, inplace = True)
x_test_df.replace(np.nan, 0, inplace = True)

2 - Plot data

In [143]:
r_filter = re.compile("r[0-9]+")

train_df["total_r0-52"] = list(map(
    composed_bps_returns,
    train_df[list(filter(r_filter.match, train_df.columns))].to_numpy()
    ))

In [144]:
#plot_matrix(train_df, "day", "equity", "total_r0-52", lambda x : list(x)[0])

# Add features
1 - Time features (day of the week)

In [145]:
train_df["week_day"] = train_df["day"].apply(lambda x : x % 5)
#plot_matrix(train_df,"week_day", "equity", "total_r0-52", lambda x : list(x)[0], x_lim=20)

# Preprocess

In [146]:
features_columns = list(filter(r_filter.match, train_df.columns)) # + ["week_day", "total_r0-52"]
preprocessed_features_columns = [f"preprocessed_{col}" for col in features_columns]

In [147]:
train_df.sort_values(by="day", inplace= True) # Sort by day to get as close as possible from the test dataset conditions with my validation dataset

train_ratio = 0.8
train_limit = int(train_ratio*len(train_df))

X_train = train_df[features_columns].iloc[:train_limit].to_numpy()
y_train = tf.keras.utils.to_categorical(train_df['reod'].iloc[:train_limit], num_classes=3)

X_validation = train_df[features_columns].iloc[train_limit:].to_numpy()
y_validation = tf.keras.utils.to_categorical(train_df['reod'].iloc[train_limit:], num_classes=3)

X_test = x_test_df[features_columns].to_numpy()

# Model

In [149]:

def reccurent_layer(type, *args, **kwargs):
    if type == "LSTM":
        return tf.keras.layers.LSTM(*args, **kwargs)
    elif type == "GRU":
        return tf.keras.layers.GRU(*args, **kwargs)
    raise ValueError(f"Type args is wrong : '{type}' while it should be 'LSTM' or 'GRU'")

def optimizer(type, *args, **kwargs):
    if type == "adam":
        return tf.keras.optimizers.Adam(*args, **kwargs)
    elif type == "rmsprop":
        return tf.keras.optimizers.RMSprop(*args, **kwargs)
    raise ValueError(f"Type args is wrong : '{type}' while it should be 'adam' or 'rmsprop'")


class MyHyperModel(keras_tuner.HyperModel):
    def __init__(self, X_train, y_train, X_validation, y_validation, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.X_train = X_train
        self.X_validation = X_validation
        self.y_train = y_train
        self.y_validation = y_validation

        self.tranformer_robust = RobustScaler(unit_variance = True, with_centering= True).fit(X_train)
        self.tranformer_quantile = QuantileTransformer(output_distribution='normal').fit(X_train)
        self.tranformer_power = PowerTransformer(method='yeo-johnson', standardize=True).fit(X_train)

        self.X_train_robust = self.tranformer_robust.transform(X_train)[:, :, np.newaxis]
        self.X_train_quantile = self.tranformer_quantile.transform(X_train)[:, :, np.newaxis]
        self.X_train_power = self.tranformer_power.transform(X_train)[:, :, np.newaxis]
        self.X_validation_robust = self.tranformer_robust.transform(X_validation)[:, :, np.newaxis]
        self.X_validation_quantile = self.tranformer_quantile.transform(X_validation)[:, :, np.newaxis]
        self.X_validation_power = self.tranformer_power.transform(X_validation)[:, :, np.newaxis]



    
    def build(self, hp):
        nb_reccurrent_layers = hp.Int("nb_reccurrent_layers", min_value = 1, max_value = 4, step=1)
        reccurrent_unit = hp.Int("reccurrent_unit", min_value = 4, max_value = 512, sampling = "log")
        reccurrent_dropout = hp.Float("reccurrent_dropout", min_value = 0, max_value = 0.3, sampling = "linear")
        reccurent_type = hp.Choice("reccurent_type", ["LSTM", "GRU"])
        attention_layer = hp.Boolean("attention_layer")
        learning_rate = hp.Float("learning_rate",  1E-5, 1E-2, sampling = "log")
        opt_type = hp.Choice("optimizer", ["adam", "rmsprop"])
        
        layers = []
        
        for i in range(nb_reccurrent_layers - 1):
            layers.append(reccurent_layer(reccurent_type, reccurrent_unit, dropout = reccurrent_dropout, return_sequences = True))
        
        if attention_layer:layers.append(AttentionLayer(reccurrent_unit, dropout = reccurrent_dropout))
        else:layers.append(reccurent_layer(reccurent_type, reccurrent_unit, dropout = reccurrent_dropout, return_sequences = False))

        
        layers.append(tf.keras.layers.Dense(3, activation= "softmax"))

        model = tf.keras.models.Sequential(layers)
        
        model.compile(
            optimizer = optimizer(opt_type, learning_rate = learning_rate),
            loss = hp.Choice("loss", ["categorical_crossentropy", "categorical_hinge"]), 
            metrics=["acc"]
        )
        
        return model
    
    
    def fit(self, hp, model, *args, **kwargs):
        preprocessing_method = hp.Choice("preprocess", ["robust_scale", "power_transform", "quantile_transform", "none"])
        if preprocessing_method == "robust_scale":
            X_train = self.X_train_robust
            X_validation = self.X_validation_robust
        elif preprocessing_method == "quantile_transform":
            X_train = self.X_train_quantile
            X_validation = self.X_validation_quantile
        elif preprocessing_method == "power_transform":
            X_train = self.X_train_power
            X_validation = self.X_validation_power
        else:
            raise ValueError(f"Preprocessing method is invalid : '{preprocessing_method}'")
            
        seq_len = hp.Int("seq_len", min_value = 5, max_value = X_train.shape[1], step=1)

        
        return model.fit(
            X_train[:, -seq_len:, :], self.y_train,
            validation_data = (X_validation[:, -seq_len:, :], self.y_validation),
            *args, **kwargs
        )

In [152]:
tuner = keras_tuner.Hyperband(
    MyHyperModel(X_train, y_train, X_validation, y_validation),
    objective="val_acc",
    max_epochs=30,
    factor=2,
    hyperband_iterations=1,
    directory="keras_tuner_checkpoints",
    project_name="tune_hypermodel2",
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, mode='min')

In [153]:
tuner.search(callbacks = [stop_early])

Trial 41 Complete [00h 00m 00s]

Best val_acc So Far: 0.4814999997615814
Total elapsed time: 00h 49m 04s

Search: Running Trial #42

Value             |Best Value So Far |Hyperparameter
1                 |1                 |nb_reccurrent_layers
99                |141               |reccurrent_unit
0.21614           |0.16375           |reccurrent_dropout
LSTM              |GRU               |reccurent_type
True              |False             |attention_layer
2.0841e-05        |0.0025691         |learning_rate
rmsprop           |rmsprop           |optimizer
categorical_hinge |categorical_cro...|loss
power_transform   |robust_scale      |preprocess
51                |40                |seq_len
8                 |4                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |3                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/8


Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/engine/base_tuner.py", line 266, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/engine/base_tuner.py", line 231, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/tuners/hyperband.py", line 419, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/engine/tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/engine/tuner.py", line 214, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/tmp/ipykernel_32/1106451357.py", line 86, in fit
    return model.fit(
  File "/usr

RuntimeError: Number of consecutive failures excceeded the limit of 3.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/engine/base_tuner.py", line 266, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/engine/base_tuner.py", line 231, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/tuners/hyperband.py", line 419, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/engine/tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/engine/tuner.py", line 214, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/tmp/ipykerne

In [ ]:
#import shutil
#shutil.rmtree('/notebooks/keras_tuner_checkpoints/tune_hypermodel')

# Test best models

In [ ]:
dict_best_hp = { key:[] for key in tuner.get_best_hyperparameters()[0].values.keys()}
list_best_hp = tuner.get_best_hyperparameters(num_trials= 10)
for i in range(10):
    for key in dict_best_hp.keys():
        dict_best_hp[key].append(list_best_hp[i].values[key] if key in list_best_hp[i].values.keys() else None)
pd.DataFrame(dict_best_hp)

In [164]:
tranformer_robust = RobustScaler(unit_variance = True, with_centering= True).fit(X_train)

X_train_robust = tranformer_robust.transform(X_train)[:, :, np.newaxis]
X_validation_robust = tranformer_robust.transform(X_validation)[:, :, np.newaxis]


seq_len = 40
dropout = 0.3
model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(120, return_sequences = True, dropout= dropout, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.GRU(120, return_sequences = False, dropout= dropout, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dense(3, activation = "softmax")
])
model.compile(
    loss="hinge",
    optimizer= tf.keras.optimizers.Adam(learning_rate = 2.5E-3),
    metrics = ["acc"]
)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="models_weights/Model_{epoch:02d}-{val_loss:.4f}-{val_acc:0.4f}.hdf5",
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

model.fit(
    X_train_robust[:, -seq_len:, :], y_train,
    validation_data = (X_validation_robust[:, -seq_len:, :], y_validation),
    batch_size= 64,
    epochs = 20,
    callbacks=[model_checkpoint_callback]
)

Epoch 1/20
1250/1250 [==============================] - 18s 11ms/step - loss: 1.0623 - acc: 0.4115 - val_loss: 1.0485 - val_acc: 0.4273
Epoch 2/20
1250/1250 [==============================] - 11s 9ms/step - loss: 1.0591 - acc: 0.4114 - val_loss: 1.0485 - val_acc: 0.4273
Epoch 3/20
1250/1250 [==============================] - 12s 9ms/step - loss: 1.0591 - acc: 0.4114 - val_loss: 1.0485 - val_acc: 0.4273
Epoch 4/20
1250/1250 [==============================] - 12s 10ms/step - loss: 1.0591 - acc: 0.4114 - val_loss: 1.0485 - val_acc: 0.4273
Epoch 5/20
1250/1250 [==============================] - 12s 9ms/step - loss: 1.0591 - acc: 0.4114 - val_loss: 1.0485 - val_acc: 0.4273
Epoch 6/20
 416/1250 [========>.....................] - ETA: 7s - loss: 1.0590 - acc: 0.4115

KeyboardInterrupt: 

In [ ]:
x_train_df = pd.read_csv("input_training.csv", index_col="ID", nrows = nrows)
x_train_df.replace(np.nan, 0, inplace = True)

X_train = train_df[features_columns].to_numpy()
y_train = train_df['reod']

In [ ]:
tranformer_robust = RobustScaler(unit_variance = True, with_centering= True).fit(X_train)

X_train_robust = tranformer_robust.transform(X_train)[:, :, np.newaxis]

X_test_robust = tranformer_robust.transform(X_test)[:, :, np.newaxis]

seq_len = 30
dropout = 0.123731
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(41, return_sequences = False, dropout= dropout),
    tf.keras.layers.Dense(3, activation = "softmax")
])
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer= tf.keras.optimizers.Adam(learning_rate = 3E-3),
    metrics = ["acc"]
)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="models_weights/Model_{epoch:02d}-{val_loss:.4f}-{val_acc:0.4f}.hdf5",
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

model.fit(
    X_train_robust[:, -seq_len:, :], y_train,
    batch_size= 64,
    epochs = 2,
    #callbacks=[model_checkpoint_callback]
)

In [ ]:
#model.load_weights("/notebooks/models_weights/Model_03-0.9985-0.4751.hdf5")
results = model.predict(X_test_robust[:, -seq_len:, :])

In [ ]:
x_test_df["reod"] = results.argmax(axis=-1) -1
result_df = x_test_df[["reod"]]
result_df

In [ ]:
result_df.to_csv("y_test.csv")